In [5]:
import mido
import os
import numpy as np
from scipy import stats
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from tensorflow.contrib import rnn
import time

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = (12, 8)

In [6]:
TICKS_PER_BEAT = 480
TEMPO = int(mido.bpm2tempo(120))

In [7]:
%run get_training_data.py

### Create training data

In [8]:
def get_training_data():
    """Generate training data array for all files in "midis_processed/" directory.
       
    Returns:
        Numpy array of training data.
    """
    training_data = []
    for filename in os.listdir('midis_processed/'):
        if filename.endswith(".mid"):
            print(filename)
            training_data.append(midi_to_vector('midis_processed/' + filename))
    return np.array(training_data)

In [9]:
def get_training_data(num):
    """Generate training data array for all files in "midis_processed/" directory.
       
    Returns:
        Numpy array of training data.
    """
    training_data = []
    for filename in os.listdir('midis_processed/')[:num]:
        if filename.endswith(".mid"):
            vector = midi_to_vector('midis_processed/' + filename)
            if len(vector) > 100:
                training_data.append(vector)
            else:
                print("Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
    return np.array(training_data)

In [10]:
# Set num training examples here. Note: actual number will be less, since some have errors
training_data = get_training_data(1)

In [11]:
print("Number of training examples:", len(training_data))

Number of training examples: 1


In [12]:
training_data = np.array([np.array(x) for x in training_data])
training_data[0].shape

(878, 1120)

### Parameters

In [13]:
# Parameters
#batch_size = 100
#learning_rate = 0.001
#training_iters = 10000
#n_input = 100
#display_step = 10

# number of units in RNN cell
#n_hidden = 512

# Parameters
batch_size = 3
learning_rate = 0.001
training_iters = 10000
n_input = 16
display_step = 10

# number of units in RNN cell
n_hidden = 1500

### Create log file

In [14]:
logfile_directory = 'logs/'

In [15]:
logfile_name = "Logfile (NTraining={}, NHidden={}, Batch={}, LR={}, Date={}).txt". \
    format(len(training_data), n_hidden, batch_size, learning_rate, time.strftime("%Y-%m-%d %H.%M"))
logfile_name

'Logfile (NTraining=1, NHidden=1500, Batch=3, LR=0.001, Date=2017-11-09 21.03).txt'

### Create tensor flow graph

In [16]:
vocab_size = len(training_data[0][0])
vocab_size

1120

In [17]:
# tf Graph input
x = tf.placeholder("float", [batch_size, n_input, vocab_size])
y = tf.placeholder("int32", [batch_size, n_input])

# RNN output node weights and biases
softmax_w = tf.Variable(tf.random_normal([n_hidden, vocab_size]))
softmax_b = tf.Variable(tf.random_normal([vocab_size]))

max_grad_norm = 5

In [18]:
# tf Graph generative input
x_generation = tf.placeholder("float", [1, vocab_size])

In [22]:
rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])
x_split = tf.split(x, n_input, 1)
x_split = [tf.reshape(a, [batch_size, vocab_size]) for a in x_split]

In [23]:
outputs, states = rnn.static_rnn(rnn_cell, x_split, dtype=tf.float32)
logits = [tf.nn.xw_plus_b(output, softmax_w, softmax_b) for output in outputs]
logits = tf.reshape(logits, [batch_size, n_input, vocab_size])

In [24]:
loss = tf.contrib.seq2seq.sequence_loss(logits, y, weights=tf.ones([batch_size, n_input], dtype=tf.float32), # check this
                                        average_across_timesteps=True, average_across_batch=True)
cost = tf.reduce_sum(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

## Run tensor flow session

In [25]:
session = tf.Session()

In [26]:
def get_midi_xy(midi_file_number):
    offset = np.random.randint(0, len(training_data[midi_file_number]) - n_input - 1)
    midi_x = training_data[midi_file_number][offset:offset+n_input]
    midi_y = training_data[midi_file_number][1 + offset:1 + offset+n_input].dot(range(vocab_size)).reshape(1, n_input)
    return midi_x, midi_y

In [27]:
def get_batch():
    """Generate random batches of x and y training vectors for RNN.
       
    Returns:
        Two batches of x and y vectors.
    """
    batch_x = []
    batch_y = []
    batch = np.random.choice(range(len(training_data)), batch_size, replace=True)
    for b in batch:
        midi_x, midi_y = get_midi_xy(b%len(training_data))
        batch_x.append(midi_x)
        batch_y.append(midi_y)
    batch_y = np.reshape(batch_y, [batch_size, n_input])
    return batch_x, batch_y

In [28]:
def seconds_to_minsecstr(seconds):
    minutes = seconds//60
    seconds -= 60*minutes
    if minutes < 10: minutes = "0" + str(minutes)
    if seconds < 10: seconds = "0" + str(seconds)
    return str(minutes) + ":" + str(seconds)

In [ ]:
# Create log file when started training
logfile = open(logfile_directory + logfile_name, "w")

training_loss_list = []

init = tf.global_variables_initializer()
session.run(init)
step = 0
loss_total = 0

start_time = time.time()

while step < training_iters:
    batch_x, batch_y = get_batch()

    _, loss = session.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
    loss_total += loss
    log = "Iter=" + str(step+1) + ", Average Loss={:.6f}, Time Elapsed={}".format(loss_total/display_step, 
             seconds_to_minsecstr(int(time.time()-start_time)))
    logfile.write(log)
    
    if (step+1) % display_step == 0:
        print(log)
        loss_total = 0
    training_loss_list.append(loss_total/display_step)
    step += 1

In [ ]:
logfile.close()

### Plots

In [4]:
plt.plot(training_loss_list)
plt.xtitle('Iteration')
plt.title('Training loss over iterations')
plt.show()

NameError: name 'training_loss_list' is not defined

## Generate music from trained model

In [93]:
N_GENERATED_NOTE_VECTORS = 1000
temperature = 1.0

In [ ]:
outputs_generation, states_generation = rnn.static_rnn(rnn_cell, [x_generation], dtype=tf.float32)
outputs_generation

In [ ]:
logits_generation = tf.nn.xw_plus_b(outputs_generation[0], softmax_w, softmax_b)
logits_generation

In [ ]:
#seed_note = np.zeros(vocab_size)
#seed_note[0] = 1
seed_note = training_data[int(len(training_data)*np.random.random())][0]
seed_note = np.reshape(seed_note, [1, vocab_size])

In [ ]:
generated_music = [seed_note]
for i in range(N_GENERATED_NOTE_VECTORS):
    onehot_pred = session.run(logits_generation, feed_dict={x_generation: generated_music[-1]})
    
    next_note = np.zeros(vocab_size)
    #index = np.argmax(onehot_pred, 1) # take max probability
    softmax_cdf = np.exp(onehot_pred[0]/temperature)/np.sum(np.exp(onehot_pred[0]/temperature))
    index = np.random.choice(range(vocab_size), p=softmax_cdf) # choose probabilistically
    
    next_note[index] = 1
    next_note = np.reshape(next_note, [1, vocab_size])
    
    generated_music.append(next_note)

In [ ]:
notes = np.array(generated_music).reshape(N_GENERATED_NOTE_VECTORS+1, 1124).dot(np.array(range(1124)))
notes

### Convert generated music to midi

In [ ]:
def get_track_sequences(vector_sequence):
    """Get note sequences for each track from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
       
    Returns:
        List of track vector sequences.
    """
    track_sequences = [[] for _ in range(NUM_TRACKS)]
    start_time = 0
    for vector in vector_sequence:
        index = vector.index(1)
        # vector is a time event.
        if index >= NUM_MIDI_PITCHES * 2 * NUM_TRACKS:
            time = (index - NUM_MIDI_PITCHES * 2 * NUM_TRACKS + 1) * TIMESHIFT_LENGTH
            start_time += time
        else:
            # vector is a note_off event.
            if index >= NUM_MIDI_PITCHES * NUM_TRACKS:
                note_type = 'note_off'
                index -= NUM_MIDI_PITCHES * NUM_TRACKS
            # vector is a note_on event.
            else:
                note_type = 'note_on'
            track_num = int(index / NUM_MIDI_PITCHES)
            note = index % NUM_MIDI_PITCHES
            track_sequences[track_num].append({"type": note_type, "note": note, "start_time": start_time})
            time_delay = 0
            
    return track_sequences
    

In [ ]:
def get_midi_from_vector_sequence(track_sequences):
    """Get midi from track sequences.

    Args:
       track_sequences: List of vector sequences for each track.
       
    Returns:
        MidiFile object.
    """
    mid = mido.MidiFile()
    tracks = [mido.MidiTrack() for _ in range(NUM_TRACKS)]
    mid.tracks.extend(tracks)

    for i, ts in enumerate(track_sequences):
        prev_start_time = 0
        for event in ts:
            time = event["start_time"] - prev_start_time
            prev_start_time = event["start_time"]
            # I picked a random number for velocity.
            mid.tracks[i].append(mido.Message(event["type"], note=event["note"], velocity=50,
                                              time=int(mido.second2tick(time, TICKS_PER_BEAT, TEMPO))))
    
    return mid

In [ ]:
track_sequence = get_track_sequences([list(x[0]) for x in generated_music])
track_sequence

In [ ]:
midi = get_midi_from_vector_sequence(track_sequence)
midi

# Save midi

In [ ]:
generation_output = "generated_music/"

In [97]:
midi_name = "Generated Music (NTraining={}, NHidden={}, Batch={}, LR={}, Length={}, Temperature={}, Date={}).txt". \
    format(len(training_data), n_hidden, batch_size, learning_rate, N_GENERATED_NOTE_VECTORS, temperature, time.strftime("%Y-%m-%d %H.%M"))
midi_name

'Generated Music (NTraining=1, NHidden=1500, Batch=3, LR=0.001, Length=1000, Temperature=1.5, Date=2017-11-09 21.00).txt'

In [ ]:
midi.save(generation_output + midi_name)